In [1]:
from pyscipopt import Model, quicksum

In [2]:
import random
random.seed(42) # For reproducibility

In [3]:
def print_results(model, EPS=1.e-6):
    x,y = model.data
    edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
    facilities = [j for j in y if model.getVal(y[j]) > EPS]
    print("Optimal value=", model.getObjVal())
    print("Facilities at nodes:", facilities)
    print("Edges:", edges)

# Uncapacitated Facility Location
$$
\begin{align*}
    z = \min \sum_{i=1}^n \sum_{j=1}^m c_{ij} x_{ij} + \sum_{i=1}^n f_i y_i\\
    \sum_{i=1}^n x_{ij} &= 1 \quad \forall j \in \{1, \ldots, m\}\\
    \sum_{j=1}^m x_{ij} &\leq M y_j \quad \forall i \in \{1, \ldots, n\}\\
    x &\in \{0,1\}^{n\times m}\\
    y &\in \{0,1\}^n
\end{align*}
$$

In [4]:
def uncapacitated_facility_location(facilities_number, clients_number, fixed_costs, costs, capacity):
    model = Model("Uncapacitated facility location")
    x,y = {},{}
    # Adding the variables to the model
    for j in range(facilities_number):
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in range(clients_number):
            x[i,j] = model.addVar(vtype="B", name="x(%s,%s)"%(i,j))
    for i in range(clients_number):
        model.addCons(quicksum(x[i,j] for j in range(facilities_number)) == 1, "Demand(%s)"%i)
    for j in range(facilities_number):
        model.addCons(
            quicksum(x[i,j] for i in range(clients_number)) <= capacity[j]*y[j], "Capacity(%s)"%i
        )
    model.setObjective(
        quicksum(fixed_costs[j]*y[j] for j in range(facilities_number)) +
        quicksum(costs[i,j]*x[i,j] for i in range(clients_number) for j in range(facilities_number)),
        "minimize")
    model.data = x,y
    return model

In [5]:
facilities_number = 10
clients_number = 8
min_cost = 1
max_cost = 100
default_capacity = 1
capacity = {j:default_capacity for j in range(facilities_number)}
costs = {(i,j):random.randint(min_cost, max_cost) for i in range(clients_number) for j in range(facilities_number)}
fixed_costs = {j:random.randint(min_cost, max_cost) for j in range(facilities_number)}

In [6]:
model = uncapacitated_facility_location(facilities_number, clients_number, fixed_costs, costs, capacity)
model.optimize()
print_results(model)

Optimal value= 429.0
Facilities at nodes: [0, 2, 3, 4, 5, 6, 8, 9]
Edges: [(7, 0), (0, 2), (2, 3), (5, 4), (3, 5), (4, 6), (1, 8), (6, 9)]


# P-Medians
$$
\begin{align*}
    z = \min \sum_{i=1}^n \sum_{j=1}^m c_{ij} x_{ij} \\
    \sum_{i=1}^n x_{ij} &= 1 \quad \forall j \in \{1, \ldots, m\}\\
    \sum_{i=1}^n y_{i} &= p\\
    x_{ij} &\leq y_j \quad \forall i \in \{1, \ldots, n\} \quad \forall j \in \{1, \ldots, m\}\\
    x &\in \{0,1\}^{n\times m}\\
    y &\in \{0,1\}^n
\end{align*}
$$

In [7]:
def p_medians(I, J, costs, p):
    model = Model("P-medians")
    x,y = {},{}
    # Adding the variables to the model
    for i in range(I):
        y[i] = model.addVar(vtype="B", name="y(%s)"%i) # Y variables are binary
        for j in range(J):
            x[i,j] = model.addVar(vtype="B", name="x(%s,%s)"%(i,j)) # X variables are binary
    for i in range(I):
        model.addCons(quicksum(x[i,j] for j in range(J)) == 1, "Demand(%s)"%i)
    for i in range(I):
        for j in range(J):
            model.addCons(x[i,j] <= y[j], "Open(%s,%s)"%(i,j))
    model.addCons(quicksum(y[j] for j in range(J)) <= p, "Median")
    model.setObjective(
        quicksum(costs[i,j]*x[i,j] for i in range(I) for j in range(J)),
        "minimize")
    model.data = x,y
    return model

In [8]:
I = 10
J = 5
min_cost = 1
max_cost = 100
p = 5
costs = {(i,j):random.randint(min_cost, max_cost) for i in range(I) for j in range(J)}

In [9]:
model = p_medians(I, J, costs, p)
model.optimize()
print_results(model)

Optimal value= 208.0
Facilities at nodes: [0, 1, 2, 3, 4]
Edges: [(0, 2), (1, 4), (2, 2), (3, 1), (4, 3), (5, 2), (6, 2), (7, 3), (8, 0), (9, 0)]


# P-Centers
$$
\begin{align*}
    z = \min l \\
    \sum_{j=1}^m x_{ij} &= 1 \quad \forall i \in \{1, \ldots, n\}\\
    \sum_{j=1}^m y_{j} &= p\\
    \sum_{j=1}^m c_{ij}x_{ij} &\leq l \quad \forall i \in \{1, \ldots, n\}\\
    x_{ij} &\leq y_j \quad \forall i \in \{1, \ldots, n\} \quad \forall j \in \{1, \ldots, m\}\\
    x &\in \{0,1\}^{n\times m}\\
    y &\in \{0,1\}^n
\end{align*}
$$

In [10]:
def p_centers(I, J, costs, p):
    model = Model("P-medians")
    x,y = {},{}
    # Adding the variables to the model
    l = model.addVar(vtype="C", name="l")
    for i in range(I):
        y[i] = model.addVar(vtype="B", name="y(%s)"%i) # Y variables are binary
        for j in range(J):
            x[i,j] = model.addVar(vtype="B", name="x(%s,%s)"%(i,j)) # X variables are binary
    for i in range(I):
        model.addCons(quicksum(x[i,j] for j in range(J)) == 1, "Demand(%s)"%i)
        model.addCons(quicksum(costs[i,j]*x[i,j] for j in range(J)) <= l, "Costs(%s)"%(i))
    for i in range(I):
        for j in range(J):
            model.addCons(x[i,j] <= y[j], "Open(%s,%s)"%(i,j))
    model.addCons(quicksum(y[j] for j in range(J)) <= p, "Center")
    model.setObjective(l, "minimize")
    model.data = x,y
    return model

In [11]:
I = 10
J = 5
min_cost = 1
max_cost = 100
p = 5
costs = {(i,j):random.randint(min_cost, max_cost) for i in range(I) for j in range(J)}

In [12]:
model = p_centers(I, J, costs, p)
model.optimize()
print_results(model)

Optimal value= 35.0
Facilities at nodes: [0, 1, 2, 3, 4]
Edges: [(0, 3), (1, 3), (2, 0), (3, 1), (4, 4), (5, 2), (6, 1), (7, 0), (8, 0), (9, 3)]
